<a href="https://colab.research.google.com/github/dinurisenara/Colabra/blob/main/RoBERTaSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and import dependancies

In [1]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00


Import the necessary modules

In [2]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd


Load the Sentiment Analysis for mental health dataset

In [5]:
from google.colab import files

# Upload your kaggle.json API key
files.upload()

# Move it to the correct location
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("suchintikasarkar/sentiment-analysis-for-mental-health")

print("Path to dataset files:", path)

100%|██████████| 11.1M/11.1M [00:00<00:00, 25.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/suchintikasarkar/sentiment-analysis-for-mental-health/versions/1


In [7]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/suchintikasarkar/sentiment-analysis-for-mental-health/versions/1"
os.listdir(dataset_path)


['Combined Data.csv']

In [9]:
import pandas as pd

# Define CSV file path (replace 'your_file.csv' with actual file name)
csv_file = f"{dataset_path}/Combined Data.csv"

# Load dataset
df = pd.read_csv(csv_file)

# Show first few rows
df.head()


,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [10]:
df.info()  # Check column names and data types
df.isnull().sum()  # Check for missing values
df.columns  # List column names


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53043 entries, 0 to 53042
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  53043 non-null  int64 
 1   statement   52681 non-null  object
 2   status      53043 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


Index(['Unnamed: 0', 'statement', 'status'], dtype='object')

In [11]:
# Remove missing values
df = df.dropna(subset=['statement'])
df = df.reset_index(drop=True)

# Verify removal
df.isnull().sum()


,0
Unnamed: 0,0
statement,0
status,0


In [12]:
df['status'].value_counts()


,count
status,
Normal,16343
Depression,15404
Suicidal,10652
Anxiety,3841
Bipolar,2777
Stress,2587
Personality disorder,1077


In [13]:
label_mapping = {
    "Anxiety": 0, "Depression": 1, "Neutral": 2, "PTSD": 3, "Bipolar": 4
}

# Apply label encoding
df['label'] = df['status'].map(label_mapping)

# Verify encoding
df[['status', 'label']].head()


,status,label
0,Anxiety,0.0
1,Anxiety,0.0
2,Anxiety,0.0
3,Anxiety,0.0
4,Anxiety,0.0


In [14]:
df['status'].value_counts()


,count
status,
Normal,16343
Depression,15404
Suicidal,10652
Anxiety,3841
Bipolar,2777
Stress,2587
Personality disorder,1077


In [15]:
df['label'].unique()


array([ 0., nan,  1.,  4.])

In [16]:
label_mapping = {
    "Anxiety": 0,
    "Depression": 1,
    "Normal": 2,
    "Suicidal": 3,
    "Bipolar": 4,
    "Stress": 5,
    "Personality disorder": 6
}

# Ensure all status values are correctly mapped
df['label'] = df['status'].map(label_mapping)

# Verify if there are still NaN values
print(df['label'].unique())


[0 2 1 3 5 4 6]


In [17]:
df = df.dropna(subset=['label'])  # Drop rows where 'label' is NaN
df['label'] = df['label'].astype(int)  # Convert labels to integer


In [18]:
print(df['label'].unique())  # Should show all integers (0-6)
print(df[['status', 'label']].head())  # Check if labels are correctly mapped


[0 2 1 3 5 4 6]
    status  label
0  Anxiety      0
1  Anxiety      0
2  Anxiety      0
3  Anxiety      0
4  Anxiety      0


In [19]:
df['label'].value_counts()


,count
label,
2,16343
1,15404
3,10652
0,3841
4,2777
5,2587
6,1077


In [20]:
df.sample(10)[['status', 'label']]


,status,label
46744,Bipolar,4
8238,Suicidal,3
31341,Normal,2
8408,Suicidal,3
30341,Normal,2
31751,Normal,2
28516,Normal,2
9864,Suicidal,3
31858,Normal,2
36009,Suicidal,3


In [21]:
# Convert 'status' to string (avoids mismatching issues)
df['status'] = df['status'].astype(str)

# Reapply label mapping
df['label'] = df['status'].map(label_mapping)

# Drop NaN values (if any)
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)  # Ensure labels are integers

# Check again
df.sample(10)[['status', 'label']]


,status,label
48474,Stress,5
51202,Personality disorder,6
19509,Depression,1
14628,Depression,1
21663,Suicidal,3
52498,Anxiety,0
3501,Normal,2
38699,Depression,1
11241,Depression,1
3382,Normal,2


In [22]:
from sklearn.model_selection import train_test_split

# Splitting dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['statement'], df['label'], test_size=0.2, random_state=42
)

# Verify split
print(f"Training samples: {len(train_texts)}, Testing samples: {len(test_texts)}")


Training samples: 42144, Testing samples: 10537


In [23]:
from transformers import RobertaTokenizer

# Load RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize the dataset
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]